In [1]:
import resfuncRead as rfr
import numpy as np
from Yield_calc import * 
import pandas as pd 

from tabulate import tabulate

bpar_er = rfr.getBandFunc('data/erband_R133') #reads in band data/fit for er data
bpar_nr = rfr.getBandFunc('data/nrband_R133')

ynr_mu = rfr.makeBFunc(bpar_nr[1]['mu'])
ynr_muv = np.vectorize(ynr_mu)
ynr_sig = rfr.makeBFunc(bpar_nr[1]['sig'],True)
ynr_sigv = np.vectorize(ynr_sig)

yer_mu = rfr.makeBFunc(bpar_er[1]['mu']) # sets average fit from 1st (2nd) col in data table. 
yer_muv = np.vectorize(yer_mu) #puts mean data for er in 1D array 
yer_sig = rfr.makeBFunc(bpar_er[1]['sig'],True) #sets uncertainty 
yer_sigv = np.vectorize(yer_sig)





In [2]:
s = 1
N = 10000
ER, Yield = Yield_NR(N)
Erer, Yield_er = Yield_Er(N)

#cut unwanted data 
data = np.vstack((ER,Yield)).T
x = data[data[:,1]>=0, :]
c = x[x[:,0]>=10, :]
g = c[c[:,0]<=150, :]

Yield_nr = g[:,1]
Enr = g[:,0]


#define bands for nuclear recoil 
upper = ynr_mu(Enr)+s*ynr_sigv(Enr)
lower = ynr_mu(Enr)-s*ynr_sigv(Enr)


Data = np.vstack((Enr,Yield_nr,upper,lower)).T
Data1 = Data[np.argsort(Data[:, 0])]  #sorting data based on recoil energy. 



In [8]:
#binning data. 
df = pd.DataFrame(Data1)

bins  = [10,13.4,18.1,24.5,33.1,44.8,60.6,80.2,110,150]

df1 = pd.cut(df[0],bins, labels = False)


y = np.array(df[0].loc[df1 == 0])

print(y)



[10.00974431 10.00996995 10.02096174 10.02453513 10.02719725 10.0293304
 10.03082161 10.03750765 10.05781163 10.058972   10.0718089  10.07273422
 10.07412941 10.07806507 10.08868015 10.08883089 10.09147152 10.09294244
 10.09704624 10.1006533  10.1106365  10.11865525 10.14324662 10.17135912
 10.18192478 10.18276908 10.22811966 10.22869705 10.23117773 10.23506069
 10.24301643 10.24316654 10.25077207 10.25496666 10.26656527 10.26810805
 10.27154663 10.27307464 10.27851886 10.27897955 10.28593209 10.29036732
 10.29537839 10.30283027 10.32429342 10.33385682 10.33389584 10.33487017
 10.3584447  10.36128717 10.36414047 10.37074578 10.37189754 10.37604766
 10.37881948 10.38284085 10.39112925 10.39328931 10.40981379 10.41122833
 10.41703483 10.42048885 10.42221257 10.42278719 10.43788753 10.44294146
 10.4499003  10.45908243 10.4672655  10.47452083 10.48145561 10.48625795
 10.48851636 10.48976856 10.49094404 10.51033648 10.51072594 10.52104289
 10.53404346 10.53701296 10.54155608 10.54260262 10.

In [4]:
Percent = []

for q in np.arange(0,9):
    
    count = []
    num = []
    count1 = []
    num1 = []
    
    
    x = np.array(df[1].loc[df1 == q]) #yield
    y = np.array(df[2].loc[df1 == q]) #upper band
    z = np.array(df[3].loc[df1 == q]) #lower band 
    
    
    for i,j in zip(x,y):
        if i > j:
            n = 1
        else: 
            n = 0 

        count.append(n)
    #print(count)

    for i in count: 
        if i == 1:
            num.append(i) 


    for i,j in zip(x,z):
        if i < j:
            n = 1
        else: 
            n = 0 

        count1.append(n)
    #print(count)

    for i in count: 
        if i == 1:
            num1.append(i)


    N= len(x)
    percent = 100*(N - np.abs(len(num)+len(num1)))/N

    Percent.append(percent)


In [5]:
bin_spacing = '10-13.4','13.4-18.1','18.1-24.5','24.5-33.1','33.1-44.8','44.8-60.6','60.6-80.2','80.2-110','110-150'

In [6]:
print("Bin Spacing (keV)", '\t', "Percent in",s,"sigma")     #table column headings
print("--------------", '\t', '\t' "-------------------")

for x,y in zip(bin_spacing,Percent):
    print(x, '\t','\t', '{0:1.2f}'.format(y))


Bin Spacing (keV) 	 Percent in 1 sigma
-------------- 	 	-------------------
10-13.4 	 	 73.83
13.4-18.1 	 	 79.65
18.1-24.5 	 	 79.52
24.5-33.1 	 	 83.38
33.1-44.8 	 	 84.47
44.8-60.6 	 	 86.35
60.6-80.2 	 	 82.30
80.2-110 	 	 77.05
110-150 	 	 73.61
